# Collision Avoidance - Data Collection

If you ran through the basic motion notebook, hopefully you're enjoying how easy it can be to make your Jetbot move around! Thats very cool!  But what's even cooler, is making JetBot move around all by itself!  

This is a super hard task, that has many different approaches but the whole problem is usually broken down into easier sub-problems.  It could be argued that one of the most
important sub-problems to solve, is the problem of preventing the robot from entering dangerous situations!  We're calling this *collision avoidance*. 

In this set of notebooks, we're going to attempt to solve the problem using deep learning and a single, very versatile, sensor: the camera.  You'll see how with a neural network, camera, and the NVIDIA Jetson Nano, we can teach the robot a very useful behavior!

The approach we take to avoiding collisions is to create a virtual "safety bubble" around the robot.  Within this safety bubble, the robot is able to spin in a circle without hitting any objects (or other dangerous situations like falling off a ledge).  


Of course, the robot is limited by what's in it's field of vision, and we can't prevent objects from being placed behind the robot, etc.  But we can prevent the robot from entering these scenarios itself.

The way we'll do this is super simple:  

First, we'll manually place the robot in scenarios where it's "safety bubble" is violated, and label these scenarios ``blocked``.  We save a snapshot of what the robot sees along with this label.

Second, we'll manually place the robot in scenarios where it's safe to move forward a bit, and label these scenarios ``free``.  Likewise, we save a snapshot along with this label.

That's all that we'll do in this notebook; data collection.  Once we have lots of images and labels, we'll upload this data to a GPU enabled machine where we'll *train* a neural network to predict whether the robot's safety bubble is being violated based off of the image it sees.  We'll use this to implement a simple collision avoidance behavior in the end :)

> IMPORTANT NOTE:  When JetBot spins in place, it actually spins about the center between the two wheels, not the center of the robot chassis itself.  This is an important detail to remember when you're trying to estimate whether the robot's safety bubble is violated or not.  But don't worry, you don't have to be exact. If in doubt it's better to lean on the cautious side (a big safety bubble).  We want to make sure JetBot doesn't enter a scenario that it couldn't get out of by turning in place.

In [ ]:
import os
import cv2
import sys
sys.path.append("/usr/local/lib")
import IPython
import traitlets
import threading
import numpy as np
import pyrealsense2 as rs
import ipywidgets.widgets as widgets
from uuid import uuid1
from IPython.display import display

In [ ]:
blocked_dir = 'dataset_0512/blocked'
free_dir = 'dataset_0512/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))
image = widgets.Image(format='jpeg', width=1280, height=480) 
out = widgets.Output()

In [ ]:
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)
        with out:
            print(type(image.value))
            print(image_path)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    with out:
        print("free")
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    with out:
        print("blocked")
    
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))
display(out)

Image(value=b'', format='jpeg', height='480', width='1280')

Output()

### Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [ ]:
def rs_on():
    pipeline = rs.pipeline()
    config = rs.config()
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 6)
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 6)
    pipeline.start(config)



    try:
        while True:
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

             # Stack both images horizontally
            images = np.hstack((color_image, depth_colormap))

            # Show images
            save_image = cv2.resize(color_image, (224, 224), interpolation=cv2.INTER_CUBIC)
            image.value = cv2.imencode('.jpg',save_image)[1].tobytes()
            #image.value = cv2.imencode('.jpg',images)[1].tobytes()

    finally:

        # Stop streaming
        pipeline.stop()

thread =threading.Thread(target=rs_on)
thread.start()

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!